# Starter - Swift and Python

**⚠️ This is the starter version, for you to code along with live.**

In this example, we're going to add some Python magic to the the multilayer peceptron XOR network that we made in the previous activity.


## Setting up

First, we need to `import` the TensorFlow framework:

In [0]:
import TensorFlow

Next, we need to `import Python`, and hook it into the notebook environment:

In [0]:
// code goes here

('inline', 'module://ipykernel.pylab.backend_inline')


## Creating the model

As before, we'll create our model:

In [0]:
// Create a XORModel Struct
struct XORModel: Layer
{
  // define three layers, each of Dense type
  var inputLayer = Dense<Float>(inputSize: 2, outputSize: 2, activation: sigmoid)
  var hiddenLayer = Dense<Float>(inputSize: 2, outputSize: 2, activation: sigmoid)
  var outputLayer = Dense<Float>(inputSize: 2, outputSize: 1, activation: sigmoid)
  
  // procide the differentiable thingo
  @differentiable func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float>
  {
    return input.sequenced(through: inputLayer, hiddenLayer, outputLayer)
  }
}

## Preparing to train the model

Likewise, as before, we'll create an instance of the model, an optimiser, and some data.

In [0]:
// create an instance of our XORModel Struct (defined above)
var model = XORModel()

// create an optimizer (standard gradient descent)
let optimizer = SGD(for: model, learningRate: 0.02)

// create some training data
let trainingData: Tensor<Float> = [[0, 0], [0, 1], [1, 0], [1, 1]]

// label the training data (so we know the correct outputs)
let trainingLabels: Tensor<Float> = [[0], [1], [1], [0]]

But we'll also create an array to store our loss in, so we can keep track of it:

In [0]:
var losses: [Float] = []

Then we'll train:

In [0]:
let epochs = 100_000

In [0]:
for epoch in 0 ..< epochs
{
    // do the ting
    let 𝛁model = model.gradient { model -> Tensor<Float> in
        let ŷ = model(trainingData)
        let loss = meanSquaredError(predicted: ŷ, expected: trainingLabels)
        if epoch % 5000 == 0
        {
          print("epoch: \(epoch) loss: \(loss)")
        }
        losses.append(loss.scalarized())
        return loss
    }
    optimizer.update(&model, along: 𝛁model)
}

epoch: 0 loss: 0.25536832
epoch: 5000 loss: 0.2501201
epoch: 10000 loss: 0.25008714
epoch: 15000 loss: 0.25006086
epoch: 20000 loss: 0.25003833
epoch: 25000 loss: 0.25001723
epoch: 30000 loss: 0.24999528
epoch: 35000 loss: 0.24997
epoch: 40000 loss: 0.24993798
epoch: 45000 loss: 0.24989387
epoch: 50000 loss: 0.24982883
epoch: 55000 loss: 0.24972737
epoch: 60000 loss: 0.24956138
epoch: 65000 loss: 0.24927837
epoch: 70000 loss: 0.24877715
epoch: 75000 loss: 0.2478496
epoch: 80000 loss: 0.24600479
epoch: 85000 loss: 0.24184218
epoch: 90000 loss: 0.23113453
epoch: 95000 loss: 0.20894165


## Visualising with Python

We're going to use the ever-useful [Matplotlib](https://matplotlib.org/) to visualise our losses. Matplotlib is a Python library, not a Swift library.

First, we need a handle on [`matplotlib.pyplot'](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot), which is the MATLAB style Python API that Matplotlib has, providing for simple programmatic plot

In [0]:
// code goes here

## Testing the model

In [0]:
print(round(model.inferring(from: [[0, 0], [0, 1], [1, 0], [1, 1]])))

[[0.0],
 [1.0],
 [1.0],
 [1.0]]


## Python Interopability in Depth

This section is an up-to-date derivative of some of the TensorFlow documentation.

You can ask for the Python version:

In [0]:
print(Python.version)

You can also _set_ a specific Python version, if you need. 

Note: you should run `PythonLibrary`.useVersion right after import Python, before calling any Python code. It cannot be used to dynamically switch Python versions. The Swift class [`PythonLibrary`](https://www.tensorflow.org/swift/api_docs/Structs/PythonLibrary) represents... a Python library!

In [0]:
// PythonLibrary.useVersion(2)
// PythonLibrary.useVersion(3, 7)

Using Swift, you can represent an object from Python using the Swift class [`PythonObject`](https://www.tensorflow.org/swift/api_docs/Structs/PythonObject). Everything Python will return a Swift `PythonObject`.

All of Swift's basic types can be converted to a `PythonObject`. Some happen implicitly, and some need to be cast from a Swift value to a `PythonObject` using a `PythonObject` initialiser:




In [0]:
let pythonInt: PythonObject = 1
let pythonFloat: PythonObject = 3.0
let pythonString: PythonObject = "Hello Python!"
let pythonRange: PythonObject = PythonObject(5..<10)
let pythonArray: PythonObject = [1, 2, 3, 4]
let pythonDict: PythonObject = ["foo": [0], "bar": [1, 2, 3]]

To make it easier to use in Swift, `PythonObject` defines most standard operations, including numeric operations, indexing, and iteration:

In [0]:
print(pythonInt + pythonFloat)
print(pythonString[0..<6])
print(pythonRange)
print(pythonArray[2])
print(pythonDict["bar"])

You can also convert a `PythonObject` back to Swift types:

In [0]:
let int = Int(pythonInt)!
let float = Float(pythonFloat)!
let string = String(pythonString)!
let range = Range<Int>(pythonRange)!
let array: [Int] = Array(pythonArray)!
let dict: [String: [Int]] = Dictionary(pythonDict)!

And you can, of course, perform all the operations that you'd expect. The outputs are, also of course, the same as from Python:

In [0]:
print(Float(int) + float)
print(string.prefix(6))
print(range)
print(array[2])
print(dict["bar"]!)

You need to be careful when you're using Python and Swift together, as the compiler can't figure out things about Python objects. For example, if you have a `PythonObject` that holds a Python String:

In [0]:
var myPythonString: PythonObject = "I am a Python String!"
print(myPythonString)

I am a Python String!


And then, for example, try and add 5 to it. The compiler won't complain, but it will crash (which could be 10 hours into a complex training run) on runtime:

In [0]:
// myPythonString = myPythonString + 5 // this will crash on runtime, but compile just fine
print(myPythonString)

`PythonObject` defines conformances to many useful, standard Swift protocols: 

* [Equatable](https://developer.apple.com/documentation/swift/equatable) 
* [Comparable](https://developer.apple.com/documentation/swift/comparable) 
* [Hashable](https://developer.apple.com/documentation/swift/hashable) 
* [SignedNumeric](https://developer.apple.com/documentation/swift/numeric) 
* [Strideable](https://developer.apple.com/documentation/swift/strideable) 
* [MutableCollection](https://developer.apple.com/documentation/swift/mutablecollection) 
* The [ExpressibleBy_Literal](https://developer.apple.com/documentation/swift/swift_standard_library/initialization_with_literals) protocols

You can learn more about this in [the documentation](https://www.tensorflow.org/swift/api_docs/Structs/PythonObject). None of the conformances type-safe, and crashes will occur if you attempt to use protocol functionality from an incompatible PythonObject instance.

In [0]:
let one: PythonObject = 1
print(one == one)
print(one < one)
print(one + one)

let array: PythonObject = [1, 2, 3]
for (i, x) in array.enumerated() {
    print(i, x)
}

When you convert a tuple from Python to Swift, you have to statically know the arity of the tuple and call one of the instance methods `PythonObject.tuple2`, `PythonObject.tuple3`, or `PythonObject.tuple4`.

In [0]:
let pythonTuple = Python.tuple([1, 2, 3])
print(pythonTuple, Python.len(pythonTuple))

// Convert to Swift.
let tuple = pythonTuple.tuple3
print(tuple)

You an also use `Python.builtins` to access all the Python builtins:

In [0]:
_ = Python.builtins

print(Python.type(1))
print(Python.len([1, 2, 3]))
print(Python.sum([1, 2, 3]))

And, as we showed above with Matplotlib, you can access and import Python modules:

In [0]:
let np = Python.import("numpy")
print(np)
let zeros = np.ones([2, 3])
print(zeros)

There's even support for checking that the Python import is safe:

In [0]:
let maybeModule = try? Python.attemptImport("nonexistent_module")
print(maybeModule)

There's also explicit support for converting the Swift types `Array<Element>`, `ShapedArray<Scalar>`, and  `Tensor<Scalar>` to and from Python's `numpy.ndarray`.

Note that the conversion will only succeed if the `dtype` of the `numpy.ndarray` is compatible with the `Element` or `Scalar` generic parameter type.

For `Array`, conversion from numpy succeeds only if the `numpy.ndarray` is one dimension (1-D).

In [0]:
let numpyArray = np.ones([4], dtype: np.float32)
print("Swift type:", type(of: numpyArray))
print("Python type:", Python.type(numpyArray))
print(numpyArray.shape)

Examples of converting `numpy.ndarray` to Swift types:

In [0]:
let array: [Float] = Array(numpy: numpyArray)!
let shapedArray = ShapedArray<Float>(numpy: numpyArray)!
let tensor = Tensor<Float>(numpy: numpyArray)!

Examples of converting Swift types to `numpy.ndarray`:

In [0]:
print(array.makeNumpyArray())
print(shapedArray.makeNumpyArray())
print(tensor.makeNumpyArray())

Examples with different `dtype`:

In [0]:
let doubleArray: [Double] = Array(numpy: np.ones([3], dtype: np.float))!
let intTensor = Tensor<Int32>(numpy: np.ones([2, 3], dtype: np.int32))!